In [ ]:
import os
os.environ['AWS_PROFILE'] = 'admin'
os.environ['HAVEN_DATABASE'] = 'haven'

from mirrorverse.utils import read_data_w_cache
import geopy.distance
import h3
import numpy as np
from tqdm import tqdm
import pandas as pd
import plotly.express as px

In [ ]:
data = read_data_w_cache(
    '''
    select 
        *
    from 
        plankton_velocity
    where 
        year = 2015
        and month = 1
        and depth = -25
    '''
)
print(data.shape)
data.head()

In [ ]:
data['lat'] = data['h3_index'].apply(lambda h: h3.h3_to_geo(h)[0])
data['lon'] = data['h3_index'].apply(lambda h: h3.h3_to_geo(h)[1])
data.head()

In [ ]:
lon_res = 0.1
lat_res = 0.1

data['lat_bin'] = (data['lat'] // lat_res)
data['lon_bin'] = (data['lon'] // lon_res)
data['lat'] = (data['lat'] // lat_res) * lat_res + lat_res/2
data['lon'] = (data['lon'] // lon_res) * lon_res + lon_res/2
unique_bins = set([
    (row['lat'], row['lon']) for _, row in data.iterrows()
])
len(unique_bins)

In [ ]:
rows = []
res = 0.1
for bin in unique_bins:
    lat_p = (bin[0] + res, bin[1])
    lon_p = (bin[0], bin[1] + res)
    lat_d = geopy.distance.geodesic(bin, lat_p).km 
    lon_d = geopy.distance.geodesic(bin, lon_p).km
    rows.append({
        'lat': bin[0],
        'lon': bin[1],
        'dlat': res / lat_d,
        'dlon': res / lon_d
    })

conversions = pd.DataFrame(rows)
data = data.merge(conversions)
data.head()

In [ ]:
data['velocity_east'] = data['u'] * data['dlon'] * 3600 / 1000
data['velocity_north'] = data['v'] * data['dlat'] * 3600 / 1000

In [ ]:
data = data.groupby(['lat', 'lon', 'lat_bin', 'lon_bin'])[['velocity_east', 'velocity_north']].mean().reset_index()

In [ ]:
px.scatter(data, x='lon', y='lat', color='velocity_north')

In [ ]:
px.scatter(data, x='lon', y='lat', color='velocity_east')

In [ ]:
velocities = {
    (row['lat_bin'], row['lon_bin']): (row['velocity_north'], row['velocity_east'])
    for _, row in data.iterrows()
}
x = np.array(sorted(data['lon_bin'].unique()))
y = np.array(sorted(data['lat_bin'].unique()))

U = np.zeros([x.shape[0], y.shape[0]])
V = np.zeros([x.shape[0], y.shape[0]])
for i in range(U.shape[0]):
    for j in range(U.shape[1]):
        lon_bin = x[i]
        lat_bin = y[j]
        v, u = velocities.get((lat_bin, lon_bin), (0, 0))
        U[i,j] = u 
        V[i,j] = v



x = np.array(sorted(data['lon'].unique())).astype(float)
y = np.array(sorted(data['lat'].unique())).astype(float)
t = np.arange(0, 24, 1).astype(float)
#t = np.arange(-23, 1, 1).astype(float)

U = np.tile(U, (t.shape[0], 1, 1))
V = np.tile(V, (t.shape[0], 1, 1))

In [ ]:
from numbacs.flows import get_interp_arrays_2D, get_flow_2D
from numbacs.integration import flowmap_grid_2D
from numbacs.diagnostics import ftle_grid_2D

T = 23
params = np.array([1.0])
#T = -23
#params = np.array([-1.0])

grid_vel, C_eval_u, C_eval_v = get_interp_arrays_2D(t, x, y, U, V)
funcptr = get_flow_2D(grid_vel, C_eval_u, C_eval_v, extrap_mode='linear')
flowmap = flowmap_grid_2D(funcptr, 0, T, x, y, params)
ftle = ftle_grid_2D(flowmap, T, lon_res, lat_res)

In [ ]:
import matplotlib.pyplot as plt


fig,ax = plt.subplots()
ax.contourf(x,y,ftle.T,levels=100)
ax.set_aspect('equal')

In [ ]:
h3_indices = read_data_w_cache(
    '''
    select distinct
        h3_index
    from 
        plankton_velocity
    where 
        year = 1995
        and month = 5
        and depth = -25
    '''
)
print(h3_indices.shape)
h3_indices.head()

In [ ]:
y_mesh, x_mesh = np.meshgrid(y, x)
df = pd.DataFrame({
    'lat': y_mesh.flatten(),
    'lon': x_mesh.flatten(),
    'exp': ftle.flatten()
})

df['h3_index'] = df.apply(lambda x: h3.geo_to_h3(x['lat'], x['lon'], 6), axis=1)
df = df.merge(h3_indices)
df['h3_index'] = df.apply(lambda x: h3.geo_to_h3(x['lat'], x['lon'], 5), axis=1)

df = df.groupby('h3_index')['exp'].mean().reset_index()

print(df.shape)

df.head()

In [ ]:
from mirrorverse.plotting import build_geojson
import plotly.graph_objects as go

fig = go.Figure()

geojson = build_geojson(df, 'h3_index')
fig.add_trace(
    go.Choroplethmapbox(
        geojson=geojson,
        locations=df['h3_index'],
        z=df['exp'],
        visible=True,
        marker_line_color='rgba(255,255,255,0)',
    )
)

fig.update_layout(
    autosize=False, # Disable autosizing
    width=800, # Set width in pixels
    height=800, # Set height in pixels
)

fig.update_layout(
    margin={"r":0,"t":30,"l":0,"b":0}, mapbox=dict(style="carto-positron", zoom=4, center = {"lat": 57, "lon": -150})
)

fig.show()

In [ ]:
fig.write_html('1995_ftle.html')